In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

import gurobipy as gp
from gurobipy import GRB

import re
import warnings
warnings.filterwarnings("ignore")

# Import shared calculation functions from module
from calculations import (
    load_location_rankings,
    load_ppu_data,
    categorize_ppus,
    create_ppu_quantities,
    create_renewable_ppu_tracking,
    calculate_max_capacity,
    create_storage_tracking,
    create_incidence_tracking,
    load_cost_data,
    get_component_data,
    calculate_chain_efficiency,
    calculate_chain_cost,
    calculate_ppu_metrics,
    enrich_ppu_quantities
)


# Optimal PPU Mix for Sovereign CO₂-Neutral Energy in Switzerland

## Problem Overview
This optimization problem determines the optimal mix of Power Production Units (PPUs) to achieve Switzerland's energy sovereignty with net-zero CO₂ emissions over an 80-year horizon. The model balances cost minimization, renewable energy maximization, and reduced external energy dependencies through a portfolio-style objective that penalizes expensive, carbon-intensive, and sovereignty-risk energy sources.

---

## Sets and Indices
- **T**: Set of time slices (e.g., 15-minute intervals over a year), indexed by $t \in T$.
- **K**: Set of PPU types (technologies), indexed by $k \in K$.
- **M**: Set of storage types, indexed by $m \in M$.

---

## Parameters
- **Demand and Costs**:
  - $D_t$: Electricity demand in time slice $t$ [kWh].
  - $p_{t,k}$: Delivered cost proxy for PPU $k$ in slice $t$ [CHF/kWh] (e.g., LCOE + grid adder).
  - $e_{t,k}$: Emissions factor for PPU $k$ in slice $t$ [kgCO₂e/kWh].
  - $s_{t,k}$: Sovereignty penalty for PPU $k$ in slice $t$ (dimensionless; larger values indicate less sovereign/more import-reliant energy).
- **Weights and Safeguards**:
  - $\alpha, \beta, \gamma \ge 0$: User-chosen weights for price, emissions, and sovereignty penalty.
  - $\varepsilon > 0$: Small numerical safeguard (e.g., $10^{-6}$ times the median cost) to avoid division by zero.
- **Storage Capacities**:
  - $\text{value}_m$: Storage capacity per GW-PPU for storage type $m$ [MWh/GW-PPU].
  - $k(m)$: The PPU type $k$ that provides storage $m$ (assuming a one-to-one mapping for simplicity; e.g., Lake storage provided by HYD\_S PPU).

---

## Decision Variables
- $x_k \ge 0$: Number of GW of PPU type $k$ to deploy (can be integer or continuous depending on implementation).
- $V_{t,k} \ge 0$: Energy delivered by PPU $k$ during time slice $t$ [kWh].
- $S_{t,m} \ge 0$: Storage level of storage type $m$ at the end of time slice $t$ [MWh] (optional: if storage dynamics are modeled explicitly).

---

## Problem Statement — Aggregate Period Cost and Return (with Demand Balance)
This section defines a portfolio-style objective that maximizes return slice-by-slice over time while meeting demand at every instant. It treats each technology as an asset and penalizes expensive, carbon-intensive, and sovereignty-risk energy.

### Definitions (time- and tech-indexed)
Let $T$ be the set of time slices (e.g., 15-minute intervals), indexed by $t$.  
Let $K$ be the set of technologies / PPUs, indexed by $k$.

- $V_{t,k}$ [kWh]: energy delivered by technology $k$ during slice $t$.
- $p_{t,k}$ [CHF/kWh]: delivered cost proxy (e.g., LCOE + grid adder).
- $e_{t,k}$ [kgCO$_2$e/kWh]: emissions factor.
- $s_{t,k}$ (dimensionless): sovereignty penalty per kWh (larger = less sovereign / more import reliance / lower firmness, etc.).
- $\alpha,\,\beta,\,\gamma \ge 0$: user-chosen weights for price, emissions, and sovereignty penalty.
- $\varepsilon > 0$: small numerical safeguard (e.g., $10^{-6}$ times the median cost) to avoid division by zero.
- $D_t$ [kWh]: electricity demand in slice $t$.

### Composite period cost
We define the composite cost in each slice as a weighted sum of price, emissions, and sovereignty penalty:

$$
\mathrm{cost}_t \;=\; \alpha \sum_{k\in K} p_{t,k} V_{t,k}
\;+\; \beta \sum_{k\in K} e_{t,k} V_{t,k}
\;+\; \gamma \sum_{k\in K} s_{t,k} V_{t,k}.
$$

- First term: monetary expenditure.  
- Second term: environmental externality (can be constrained or priced via $\beta$).  
- Third term: exposure to foreign / unsovereign energy (penalized with $\gamma$).

### Period return (portfolio-style)
Define the return of a slice as the inverse of its composite cost:

$$
\mathrm{Return}_t \;=\; \frac{1}{\mathrm{cost}_t + \varepsilon} \, .
$$

Maximizing $\sum_t \frac{1}{\mathrm{cost}_t + \varepsilon}$ emphasizes the harmonic mean of costs: it penalizes spikes in expensive periods more than a simple arithmetic average would. This matches the risk preference of a power system planner who wants to avoid exposure to high-price scarcity hours. The objective thus pushes the portfolio toward technologies and schedules that keep every slice affordable, not just the average.

### Objective — maximize total return over the year
$$
\max \; \sum_{t\in T} \frac{1}{\mathrm{cost}_t + \varepsilon} \, .
$$

This is equivalent in spirit to “minimize per-slice costs,” but with added downside protection against high-cost intervals.

### Demand balance (must hold every instant)
Electricity must match demand in every time slice:

$$
\sum_{k\in K} V_{t,k} \;=\; D_t \quad \forall\, t\in T \, .
$$

(The above definitions and equations can be used directly when translating the problem into a mathematical programming model or into code for numerical optimization.)

---

## Constraints (Capacity, Storage, Resource, and System Targets)

### 1. Demand Balance (restated)
$$
\sum_{k \in K} V_{t,k} \;=\; D_t \quad \forall t \in T
$$

### 2. PPU Capacity Limits
The energy delivered by each PPU cannot exceed its installed capacity (assuming linear scaling with $x_k$):

$$
V_{t,k} \;\le\; \text{capacity}_{k} \cdot x_k \cdot \Delta t \quad \forall t \in T, \; k \in K
$$

where $\text{capacity}_k$ is the power capacity per GW of PPU $k$ [GW], and $\Delta t$ is the time slice duration (e.g., 0.25 hours for 15-min slices).

### 3. Storage Capacity Limits
The storage levels must not exceed the total available capacity, which scales with the deployed PPUs. The upper limit for each storage type $m$ is:

$$
S_{\max,m} \;=\; x_{k(m)} \cdot \text{value}_m \quad \forall m \in M
$$

If storage dynamics are modeled explicitly, add:

$$
0 \;\le\; S_{t,m} \;\le\; S_{\max,m} \quad \forall t \in T, \; m \in M
$$

with storage balance equations (e.g., inflows from PPUs, outflows to demand, subject to efficiencies). A general update form is:

$$
\Delta S_t \;=\; \sum \bigl(\mathrm{Inflows}_t \cdot \eta_{\mathrm{in}}\bigr) \;-\; \sum \bigl(\mathrm{Outflows}_t / \eta_{\mathrm{out}}\bigr)
$$

### 4. Renewable Resource Availability
For renewable PPUs (e.g., PV, Wind), $V_{t,k}$ is limited by incidence data:

$$
V_{t,k} \;\le\; \text{incidence}_{t,k} \cdot x_k \cdot \Delta t \quad \forall t \in T, \; k \in K_{\text{renewable}}
$$

where $\text{incidence}_{t,k}$ is the available resource (e.g., solar irradiance, wind speed) for PPU $k$ at time $t$.

### 5. Total Energy Target
The total annual energy delivered must meet Switzerland's demand target (e.g., 113 TWh/year):

$$
\sum_{t \in T} \sum_{k \in K} V_{t,k} \;\ge\; 113 \times 10^9 \quad [\text{kWh/year}]
$$

### 6. PPU Deployment Constraint
Each selected PPU type must be deployed at exactly 1 GW:

$$
x_k \;=\; 1 \quad \forall k \in K
$$

This constraint fixes the scale of each PPU unit to 1 GW, simplifying the optimization by treating PPU deployment as a binary selection (deploy or not) rather than a continuous sizing problem.

### 7. Additional Constraints (illustrative)
- **Emissions Cap**: Total annual emissions $\le$ threshold (e.g., for net-zero target).  
- **Renewable Share**: Fraction of energy from renewables $\ge$ minimum percentage.  
- **Sovereignty**: Limit on energy from import-reliant sources.  
- **Grid and Infrastructure**: Limits on total installed capacity, ramp rates, etc.

---

## Cost and Energy Governance Formulas

### 1. Energy Conversion and Efficiency Formulas

#### 1.1 Final Available Energy in the Conversion Chain
The energy output $W_n$ after a series of $n$ components, each with efficiency $\eta_i$, is the product of the initial energy $W_1$ and all individual efficiencies. Auxiliary electricity $\sum E_i$ is added to the final electric component.

$$
W_n \;=\; W_1 \cdot \eta_1 \cdot \dots \cdot \eta_n
$$

*Description:* This multiplicative chain captures losses in sequential transformations (e.g., raw energy $\rightarrow$ transformation $\rightarrow$ electrical). It assumes no parallel paths; auxiliary inputs are post-processed.

#### 1.2 Total Chain Efficiency
The overall efficiency $\eta_{tot}$ of the conversion pathway is the ratio of final to initial energy.

$$
\eta_{tot} \;=\; \frac{W_n}{W_1}
$$

*Description:* Simplifies to $\eta_{tot} = \prod_{i=1}^n \eta_i$ from Equation (1). Used to compare pathways (e.g., direct electrification vs. hydrogen intermediation), highlighting losses in storage/reversal processes.

### 2. Cost Modeling: CAPEX and OPEX Amortization

#### 2.1 Annual Payback for Capital Amortization
The constant annual payback $P_b$ amortizes CAPEX over $n$ years at interest $Z$, solving the annuity equation for zero net present value.

$$
P_b \;=\; \text{CAPEX} \cdot \frac{Z \cdot (1 + Z)^n}{(1 + Z)^n - 1}
$$

*Description:* Derived from the geometric series for loan repayment. Assumes constant annual payments; e.g., $Z = 0.02$ (2% interest). This yields the annualized capital recovery factor (CRF).

#### 2.2 Specific Cost per Energy Unit
The levelized cost contribution $C_W$ for a component combines annualized CAPEX ($P_b$) and operational expenditure (OPEX), normalized by annual energy throughput $W_y$.

$$
C_W \;=\; \frac{P_b + \text{OPEX}}{W_y}
$$

*Description:* OPEX includes maintenance/fuel; $W_y$ is in energy units (e.g., MWh/year). For chains, sum $C_W \cdot W_i$ across components to get total LCOE.

#### 2.3 Levelized Cost of Energy (LCOE)
The total delivered energy cost is the weighted sum of component contributions.

$$
\text{LCOE} \;=\; \sum_i C_{W_i} \cdot W_i
$$

*Description:* Aggregates chain-wide costs, enabling optimization of component sizing (e.g., PV area vs. storage volume). The formulation supports PPU scalability to a 1 GW dispatchable output.

---

## Storage Governance in 15-Minute Timesteps

This section outlines how each storage component's state or availability is governed over 15-minute timesteps. For each storage, we provide the governing inflows and outflows, and the components that can extract resources from it.

### Renewable and Natural Inflows (Uncontrollable)

#### Solar \[Incidence Dependent — No Storage Option]
Inflows: Irradiance from incidence curve \[Incidence]. Outflows: To PV panels.  
Extracted by: PV panels.

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; system peak power determined by installed PV (e.g., ~20 GW in the scenario).

#### Wind \[Incidence Dependent — No Storage Option]
Inflows: Wind speed from incidence curve \[Incidence]. Outflows: To wind turbines.  
Extracted by: Wind turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; system peak power determined by installed wind (e.g., ~2 GW in the scenario).

#### River \[Incidence Dependent — No Storage Option]
Inflows: River flow from incidence curve \[Incidence]. Outflows: To hydro-turbines.  
Extracted by: Hydro-turbines (run-of-river).

**Volume Limit:**
$$
S_{\max} \;=\; 0
$$
No storage; energy follows annual/seasonal flow (e.g., ~17.8 TWh/y in the scenario).

#### Lake \[Incidence + Control Dependent]
Inflows: Reservoir inflow from incidence curve \[Incidence], pumped electricity. Outflows: To hydro-turbines.  
Extracted by: Hydro-turbines (reservoir).

**Volume Limit:**
$$
S_{\max} \;=\; 977{,}778~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

### Battery and Electrical Storage

#### Battery
Inflows: Charged electricity. Outflows: Discharged electricity to inverters.  
Extracted by: Inverters (for discharge to grid).

**Volume Limit:**
$$
S_{\max} \;=\; 800~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

### Fuel and Chemical Storage

#### Fuel Tank
Inflows: Production from chains. Outflows: To ICE, gas turbines.  
Extracted by: Internal combustion engines (ICE), gas turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 141{,}320~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### H\(_2\) Storage UG 200 bar
Inflows: Imports, electrolysis production. Outflows: To fuel cells, hydrogen turbines.  
Extracted by: Fuel cells, hydrogen turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 500{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Liquid H\(_2\) Storage
Inflows: Liquefaction. Outflows: Regasification to fuel cells.  
Extracted by: Fuel cells (after regasification).

**Volume Limit:**
$$
S_{\max} \;=\; 66{,}600~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Solar Concentrator Salt (CSP)
Inflows: Solar input. Outflows: Heat extraction to steam turbines.  
Extracted by: Steam turbines (CSP).

**Volume Limit:**
$$
S_{\max} \;=\; 4{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Bio-oil
Inflows: Pyrolysis production. Outflows: To diesel engines, boilers.  
Extracted by: Diesel engines, boilers.

**Volume Limit:**
$$
S_{\max} \;=\; 21{,}600~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Palm Oil
Inflows: Imports/production. Outflows: To refineries, engines.  
Extracted by: Refineries (for biodiesel), engines.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Wood
Inflows: Harvesting/imports. Outflows: To pyrolysis plants, boilers.  
Extracted by: Pyrolysis plants (for bio-oil), boilers.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}000~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Biogas (50% CH\(_4\))
Inflows: Anaerobic digestion production. Outflows: To gas engines, turbines.  
Extracted by: Gas engines, turbines.

**Volume Limit:**
$$
S_{\max} \;=\; 60~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### CH\(_4\) Storage — 200 bar
Inflows: Sabatier production. Outflows: To gas turbines, engines.  
Extracted by: Gas turbines, engines.

**Volume Limit:**
$$
S_{\max} \;=\; 10{,}400~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

#### Ammonia Storage
Inflows: Haber–Bosch/electrolysis production. Outflows: To ammonia crackers, turbines.  
Extracted by: Ammonia crackers (for H\(_2\)), turbines (direct combustion).

**Volume Limit:**
$$
S_{\max} \;=\; 35{,}360~\mathrm{MWh}/\mathrm{GW\text{-}PPU}
$$

---

## Solution Approach
- **Mathematical Programming**: Solve using solvers like Gurobi or CPLEX, with $V_{t,k}$ as continuous variables and $x_k$ as integer or continuous.
- **Time Series Simulation**: For large $|T|$ (e.g., 35,040 for 15-min slices), use decomposition or rolling horizon methods.
- **Sensitivity Analysis**: Vary $\alpha, \beta, \gamma$ to explore trade-offs between cost, emissions, and sovereignty.

This formulation provides a complete, classroom-ready optimization problem that can be extended with additional details as needed.


In [33]:
# Vector of raw energy sources/storage components with default integer values and typical units
raw_energy_storage = [
    {"storage": "Lake", "value": 977778, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Fuel Tank", "value": 141320, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "H2 Storage UG 200bar", "value": 500000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell", "Combined cycle power plant"]},
    {"storage": "Liquid storage", "value": 66600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell"]},
    {"storage": "Solar concentrator salt", "value": 4000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Solar concentrator steam"]},
    {"storage": "Biooil", "value": 21600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "Palm oil", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE"]},
    {"storage": "Biogas (50% CH4)", "value": 60, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "CH4 storage 200bar", "value": 10400, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "Ammonia storage", "value": 35360, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Ammonia cracking"]}
]

raw_energy_incidence = [
    {"storage": "Wood", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Pyrolysis"]},
    {"storage": "River", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Solar", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["PV"]},
    {"storage": "Wind", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Wind (onshore)", "Wind (offshore)"]}
]

In [4]:
# ============================================================================
# FULL MODEL SETUP: Functional Composition for Optimization Prep
# ============================================================================

import pandas as pd
import numpy as np
import networkx as nx  # Unused here but imported in original; keep for future
import gurobipy as gp  # For later optimization
from gurobipy import GRB
import re
import warnings
warnings.filterwarnings("ignore")

# Import all from calculations.py (assumes additions above are pasted)
from calculations import (
    # Existing
    load_location_rankings, load_ppu_data, categorize_ppus, create_ppu_quantities,
    create_renewable_ppu_tracking, calculate_max_capacity, create_storage_tracking,
    create_incidence_tracking, load_cost_data, get_component_data,
    calculate_chain_efficiency, calculate_chain_cost, calculate_ppu_metrics,
    enrich_ppu_quantities, get_incidence_data, update_storage, update_incidence,
    # New FP helpers
    pipeline, compose_ppu_setup, compose_storage_tracking, compose_incidence_tracking,
    compose_renewable_tracking, update_all_storages
)

# ============================================================================
# 1. HYPERPARAMETERS (Immutable Config)
# ============================================================================
def get_hyperparams():
    """Pure function: Return all hyperparameters as a frozen dict for immutability."""
    return {
        'T': np.arange(96 * 365),  # 35,040 timesteps
        'n_timesteps': 96 * 365,
        'delta_t': 0.25,  # hours
        'epsilon': 1e-6,
        'weights': {'alpha': 1.0, 'beta': 0.1, 'gamma': 0.5},
        'annual_demand_target': 113e9,  # kWh/year
        'solar_area_m2_per_gw': 10e6,  # m²
        'wind_turbines_offshore_per_gw': 100,
        'wind_turbines_onshore_per_gw': 300,
        'lake_max': 8.9e7,  # MWh
        'lake_growth': 0.005,
        'quantity_default': 10,
        'capacity_gw_default': 1.0
    }

hyperparams = get_hyperparams()
print(f"Total timesteps: {hyperparams['n_timesteps']}")
print(f"User-chosen weights: α={hyperparams['weights']['alpha']}, β={hyperparams['weights']['beta']}, γ={hyperparams['weights']['gamma']}")
print(f"Time slice duration: {hyperparams['delta_t']} hours")
print(f"Annual demand target: {hyperparams['annual_demand_target'] / 1e9:.1f} TWh")
print(f"Solar land requirement: {hyperparams['solar_area_m2_per_gw']:,.0f} m² per GW ({hyperparams['solar_area_m2_per_gw']/1e6:.1f} km²)")
print(f"Wind turbines per GW: {hyperparams['wind_turbines_offshore_per_gw']} offshore, {hyperparams['wind_turbines_onshore_per_gw']} onshore")

# ============================================================================
# 2. LOAD RAW DATA (Functional Pipeline)
# ============================================================================
def load_raw_data(ppu_fp: str, cost_fp: str, demand_fp: str, solar_fp: str, wind_fp: str):
    """Pure: Load all raw inputs into a dict of DataFrames."""
    return {
        'demand': pd.read_csv(demand_fp),
        'ppu_raw': load_ppu_data(ppu_fp),
        'cost': load_cost_data(cost_fp),
        'solar_locations': load_location_rankings('solar'),  # Assumes solar_fp is not needed; uses internal path
        'wind_locations': load_location_rankings('wind')
    }

raw_data = load_raw_data(
    ppu_fp='data/ppu_constructs_components.csv',
    cost_fp='data/cost_table_tidy.csv',
    demand_fp='data/monthly_hourly_load_values_2024.csv',
    solar_fp='data/solar_incidence_ranking.csv',  # Placeholder; adjust if needed
    wind_fp='data/wind_incidence_ranking.csv'
)
print(f"Loaded {len(raw_data['demand'])} demand rows")
print(f"Loaded {len(raw_data['ppu_raw'])} raw PPUs")
print(f"Loaded {len(raw_data['cost'])} cost components")
print(f"Available solar locations: {len(raw_data['solar_locations'])} sites")
print(f"Available wind locations: {len(raw_data['wind_locations'])} sites")

# ============================================================================
# 3. COMPOSE PPU SETUP (Using New FP Helper)
# ============================================================================
ppu_quantities_df = compose_ppu_setup(
    ppu_filepath='data/ppu_constructs_components.csv',
    cost_filepath='data/cost_table_tidy.csv',
    quantity=hyperparams['quantity_default'],
    capacity_gw=hyperparams['capacity_gw_default']
)
production, storage, total_ppus = categorize_ppus(raw_data['ppu_raw'])
print(f"\nPPU Production ({len(production)}): {', '.join(production[:5])}...")  # Truncate for brevity
print(f"PPU Storage ({len(storage)}): {', '.join(storage)}")
print(f"\nPPU Quantities DataFrame (first 5):\n{ppu_quantities_df.head()}")

# ============================================================================
# 4. RENEWABLE TRACKING & LOCATION ASSIGNMENTS (Composed)
# ============================================================================
renewable_tracking_df = compose_renewable_tracking(raw_data['ppu_raw'], ppu_quantities_df)
print(f"\nRenewable PPUs requiring location assignments:\n{renewable_tracking_df}")

# Empty assignment tracker (populate later via assignment logic)
ppu_location_assignments_df = pd.DataFrame(columns=[
    'PPU', 'Instance', 'Type', 'Latitude', 'Longitude', 'Rank', 'Energy_Potential'
])
print(f"\nLocation DataFrame (initially empty):\n{ppu_location_assignments_df.head()}")

# ============================================================================
# 5. STORAGE & INCIDENCE TRACKING (Composed)
# ============================================================================
# Raw vectors (from notebook; make immutable by copying)
raw_energy_storage = [
    {"storage": "Lake", "value": 977778, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Fuel Tank", "value": 141320, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "H2 Storage UG 200bar", "value": 500000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell", "Combined cycle power plant"]},
    {"storage": "Liquid storage", "value": 66600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Fuel Cell"]},
    {"storage": "Solar concentrator salt", "value": 4000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Solar concentrator steam"]},
    {"storage": "Biooil", "value": 21600, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE", "Gas Turbine"]},
    {"storage": "Palm oil", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["ICE"]},
    {"storage": "Biogas (50% CH4)", "value": 60, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "CH4 storage 200bar", "value": 10400, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Gas Turbine"]},
    {"storage": "Ammonia storage", "value": 35360, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Ammonia cracking"]}
]

raw_energy_incidence = [
    {"storage": "Wood", "value": 10000, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Pyrolysis"]},
    {"storage": "River", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Hydro Turb"]},
    {"storage": "Solar", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["PV"]},
    {"storage": "Wind", "value": 0, "unit": "MWh/GW-PPU", "current_value": 0, "extracted_by": ["Wind (onshore)", "Wind (offshore)"]}
]

storage_tracking_df = compose_storage_tracking(raw_energy_storage, ppu_quantities_df, raw_data['ppu_raw'])
incidence_tracking_df = compose_incidence_tracking(raw_energy_incidence, ppu_quantities_df, raw_data['ppu_raw'])
print(f"\nStorage Tracking DataFrame:\n{storage_tracking_df.head()}")
print(f"Incidence Tracking DataFrame:\n{incidence_tracking_df.head()}")

# Example update (for t=0; chain more as needed)
updated_storage, updated_incidence = update_all_storages(
    raw_energy_storage, raw_energy_incidence, updates={}, t=0
)
print(f"\nUpdated Storage (sample):\n{updated_storage[0]}")  # First item
print(f"Updated Incidence (sample):\n{updated_incidence[0]}")

# ============================================================================
# 6. SIMPLIFIED PPU CREATION (Integrated FP Style)
# ============================================================================
def create_simplified_ppu(name: str, ppu_type: str, components: list = None, cost_df: pd.DataFrame = None):
    """Pure: Create one PPU row; compute metrics if components given."""
    eff = calculate_chain_efficiency(components, cost_df) if components else 0.2  # Default fallback
    cost_data = calculate_chain_cost(components, cost_df) if components else {'total_cost': 0.15}
    qcost = cost_data['total_cost'] * hyperparams['delta_t'] if components else 0.25
    return pd.DataFrame([{
        'PPU': name, 'Type': ppu_type, 'Efficiency': eff,
        'Cost_CHF_per_kWh': cost_data['total_cost'], 'Quarter_CHF_per_kWh': qcost,
        'Components': components
    }])


Total timesteps: 35040
User-chosen weights: α=1.0, β=0.1, γ=0.5
Time slice duration: 0.25 hours
Annual demand target: 113.0 TWh
Solar land requirement: 10,000,000 m² per GW (10.0 km²)
Wind turbines per GW: 100 offshore, 300 onshore
Loaded 313978 demand rows
Loaded 26 raw PPUs
Loaded 83 cost components
Available solar locations: 1150 sites
Available wind locations: 1150 sites

PPU Production (18): HYD_S, HYD_R, THERM, PV, WD_ON...
PPU Storage (8): H2_G, H2_GL, H2_L, SYN_FT, SYN_METH, NH3_FULL, SYN_CRACK, CH4_BIO

PPU Quantities DataFrame (first 5):
     PPU        Type  Quantity  Capacity_GW  Total_Capacity_GW  Efficiency  \
0  HYD_S  Production        10          1.0               10.0    0.880000   
1  HYD_R  Production        10          1.0               10.0    0.880000   
2  THERM  Production        10          1.0               10.0    0.421443   
3     PV  Production        10          1.0               10.0    0.837401   
4  WD_ON  Production        10          1.0             

In [39]:
# ============================================================================
# MODEL INITIALIZATION: Constants, Variables, and Dictionaries
# ============================================================================

import pandas as pd
import numpy as np
import ast  # For literal_eval to convert string lists to Python lists
import functools

# Note: location & PPU utility functions have been moved to `calculations.py` and
# are imported from the notebook's first cell. Use load_location_rankings,
# load_ppu_data, categorize_ppus, create_ppu_quantities, create_renewable_ppu_tracking
# from that module.

# ==========================================================================
# 1. HYPERPARAMETERS AND GENERAL ASSUMPTIONS
# ==========================================================================

# Time 
T = np.arange(96 * 365)  # 96 timesteps/day * 365 days = 35,040 total
n_timesteps = len(T)
delta_t = 0.25  # 15 minutes = 0.25 hours

# Small numerical safeguard for division by zero
epsilon = 1e-6

# User-chosen weights for cost components
alpha = 1.0      # Price weight
beta = 0.1       # Emissions weight
gamma = 0.5      # Sovereignty penalty weight
weights = np.array([alpha, beta, gamma])

# Annual electricity demand target (Switzerland) - target for satisfying demand
annual_demand_target = 113e9  # kWh/year

# Land use constraints for renewable energy
solar_area_m2_per_gw = 10 * 1e6         # 10 km² = 10,000,000 m² per GW for solar panels
wind_turbines_offshore_per_gw = 100     # Number of offshore wind turbines needed per GW
wind_turbines_onshore_per_gw = 300      # Number of onshore wind turbines needed per GW
lake_max = 8.9e+7                       # Total energy storage in Lakes in Switzerland
lake_growth = 0.005                     # Annual growth rate of lake storage capacity       


# Load demand data for Switzerland (CH) from CSV
demand_df = pd.read_csv('data/monthly_hourly_load_values_2024.csv')
ppu_data_df = load_ppu_data('data/ppu_data.csv')
demand_df.head()

# The following lines assume the functions from calculations.py are available
solar_locations_df = load_location_rankings('solar')
wind_locations_df = load_location_rankings('wind')

# Optimization parameters



# Parameters for PPUs
production, storage, total = categorize_ppus(ppu_data_df)


# DataFrame to track which renewable PPUs are assigned to which locations
ppu_location_assignments_df = pd.DataFrame(columns=[
    'PPU',                 # Name of the PPU
    'Instance',            # Instance number (for multiple identical PPUs)
    'Type',                # 'solar', 'wind_onshore', or 'wind_offshore'
    'Latitude',            # Latitude of assigned location
    'Longitude',           # Longitude of assigned location
    'Rank',                # Rank of the assigned location (lower is better)
    'Energy_Potential'     # Potential value at the location (kWh/m²/hour for solar, m/s for wind)
])


print(f"User-chosen weights: α={alpha}, β={beta}, γ={gamma}")
print(f"Time slice duration: {delta_t} hours")
print(f"Annual demand target: {annual_demand_target / 1e9:.1f} TWh")
print(f"Solar land requirement: {solar_area_m2_per_gw:,.0f} m² per GW ({solar_area_m2_per_gw/1e6:.1f} km²)")
print(f"Wind turbines per GW: {wind_turbines_offshore_per_gw} offshore, {wind_turbines_onshore_per_gw} onshore")
print(f"Available solar locations: {len(solar_locations_df)} sites")
print(f"Available wind locations: {len(wind_locations_df)} sites")


Total timesteps: 35040
User-chosen weights: α=1.0, β=0.1, γ=0.5
Time slice duration: 0.25 hours
Annual demand target: 113.0 TWh
Solar land requirement: 10,000,000 m² per GW (10.0 km²)
Wind turbines per GW: 100 offshore, 300 onshore
Available solar locations: 1150 sites
Available wind locations: 1150 sites
Loaded 26 PPU components

PPU Production (18): HYD_S, HYD_R, THERM, PV, WD_ON, WD_OFF, THERM_G, THERM_M, H2P_G, H2P_L, SOL_SALT, SOL_STEAM, BIO_OIL_ICE, PALM_ICE, BIO_WOOD, WD_OFF, IMP_BIOG, NH3_P
PPU Storage (8): H2_G, H2_GL, H2_L, SYN_FT, SYN_METH, NH3_FULL, SYN_CRACK, CH4_BIO

PPU Quantities DataFrame (10 of each):
     PPU        Type  Quantity  Capacity_GW  Total_Capacity_GW
0  HYD_S  Production        10          1.0               10.0
1  HYD_R  Production        10          1.0               10.0
2  THERM  Production        10          1.0               10.0
3     PV  Production        10          1.0               10.0
4  WD_ON  Production        10          1.0               

### Parameters unique to the model (need to initialize each time you change assumptions)

Add cost and efficency measures to each PPU using the previously discussed rules. 

In [40]:
# ============================================================================
# COST AND EFFICIENCY CALCULATIONS FOR PPU COMPONENTS
# ============================================================================

# Note: cost & efficiency helper functions have been moved to `calculations.py`.
# Use load_cost_data, get_component_data, calculate_chain_efficiency,
# calculate_chain_cost, calculate_ppu_metrics and enrich_ppu_quantities from that module.

# Load cost data
cost_df = load_cost_data('data/cost_table_tidy.csv')
print(f"Loaded cost data for {len(cost_df)} components")

# Calculate metrics for each PPU
ppu_metrics_df = calculate_ppu_metrics(ppu_df, cost_df)
print("\nCalculated metrics for all PPUs:")
print(ppu_metrics_df[['PPU', 'Efficiency', 'Total_Cost_CHF_per_kWh', 'Cost_Per_15min_CHF']].sort_values(by='Cost_Per_15min_CHF'))

# Display the highest cost PPU for more details
highest_cost_ppu = ppu_metrics_df.loc[ppu_metrics_df['Cost_Per_15min_CHF'].idxmax()]
print(f"\nHighest cost PPU: {highest_cost_ppu['PPU']}")
print(f"Components: {highest_cost_ppu['Components']}")
print(f"Efficiency: {highest_cost_ppu['Efficiency']:.4f}")
print(f"Cost per kWh: {highest_cost_ppu['Total_Cost_CHF_per_kWh']:.4f} CHF")
print(f"Cost per 15min: {highest_cost_ppu['Cost_Per_15min_CHF']:.2f} CHF")
print(f"Energy produced in 15min at 1GW: {1e6 * 0.25 * highest_cost_ppu['Efficiency']} kWh")


Loaded cost data for 83 components

Calculated metrics for all PPUs:
            PPU  Efficiency  Total_Cost_CHF_per_kWh  Cost_Per_15min_CHF
13     PALM_ICE    0.227430                0.197399            0.049350
24    SYN_CRACK    0.509281                0.211560            0.052890
6       THERM_G    0.500000                0.223110            0.055777
2         THERM    0.421443                0.233785            0.058446
1         HYD_R    0.880000                0.239341            0.059835
0         HYD_S    0.880000                0.239341            0.059835
9         H2P_L    0.475000                0.272740            0.068185
12  BIO_OIL_ICE    0.282150                0.297053            0.074263
4         WD_ON    0.837401                0.300544            0.075136
3            PV    0.837401                0.304971            0.076243
7       THERM_M    0.500000                0.318110            0.079527
5        WD_OFF    0.837401                0.329168            0.08

### Rewritting the problem - simply

In [41]:
# ============================================================================
# SIMPLIFIED IMPLEMENTATION: HYPERPARAMETERS, DATAFRAMES, AND PPU CREATION
# ============================================================================

# Hyperparameters
T = np.arange(96 * 365)  # 96 timesteps/day * 365 days = 35,040 total
delta_t = 0.25  # 15 minutes = 0.25 hours
epsilon = 1e-6
alpha, beta, gamma = 1.0, 0.1, 0.5  # Weights for cost components
annual_demand_target = 113e9  # kWh/year
solar_area_m2_per_gw = 10 * 1e6  # 10 km² per GW
wind_turbines_offshore_per_gw = 100
wind_turbines_onshore_per_gw = 300

# DataFrames
ppu_df = pd.DataFrame(columns=['PPU', 'Type', 'Efficiency', 'Cost_CHF_per_kWh', 'Quarter_CHF_per_kWh'])
storage_df = pd.DataFrame(columns=['Storage', 'Capacity_MWh', 'Current_Value_MWh'])
location_df = pd.DataFrame(columns=['PPU', 'Type', 'Latitude', 'Longitude', 'Rank', 'Energy_Potential'])

# Function to create/update a PPU
def create_ppu(name, ppu_type, efficiency=None, cost_chf_per_kwh=None, quarter_chf_per_kwh=None, components=None):
    """Create a PPU entry. If efficiency/cost not provided, compute via components using get_ppu_metrics."""
    global ppu_df
    if (efficiency is None or cost_chf_per_kwh is None) and components is not None:
        eff, cost, qcost = get_ppu_metrics(name=name, components=components)
        efficiency = efficiency if efficiency is not None else eff
        cost_chf_per_kwh = cost_chf_per_kwh if cost_chf_per_kwh is not None else cost
        quarter_chf_per_kwh = quarter_chf_per_kwh if quarter_chf_per_kwh is not None else qcost

    new_ppu = {
        'PPU': name,
        'Type': ppu_type,
        'Efficiency': efficiency,
        'Cost_CHF_per_kWh': cost_chf_per_kwh,
        'Quarter_CHF_per_kWh': quarter_chf_per_kwh,
        'Components': components
    }
    ppu_df = pd.concat([ppu_df, pd.DataFrame([new_ppu])], ignore_index=True)
    print(f"PPU '{name}' added.")

# Minimal, efficient get_ppu_metrics that uses helper functions when possible
def get_ppu_metrics(name: str = None, components: list = None):
    """Return (efficiency, Cost_CHF_per_kWh, Quarter_CHF_per_kWh).

    - If `components` provided, compute via calculate_chain_* helpers using global cost_df.
    - Else, try to read from `ppu_df` columns ('Efficiency', 'Cost_CHF_per_kWh', 'Quarter_CHF_per_kWh').
    - Returns (None, None, None) if insufficient data.
    """
    # Prefer explicit components
    if components is not None:
        eff = calculate_chain_efficiency(components, cost_df) if 'calculate_chain_efficiency' in globals() else None
        cost_data = calculate_chain_cost(components, cost_df) if 'calculate_chain_cost' in globals() else None
        cost = cost_data['total_cost'] if cost_data is not None else None
        qcost = None
        # If name exists in ppu_df and has Quarter_CHF_per_kWh, prefer stored value
        if name is not None and 'Quarter_CHF_per_kWh' in ppu_df.columns:
            row = ppu_df[ppu_df['PPU'] == name]
            if not row.empty and pd.notna(row['Quarter_CHF_per_kWh'].iat[0]):
                qcost = float(row['Quarter_CHF_per_kWh'].iat[0])
        if qcost is None and cost is not None:
            qcost = float(cost) * delta_t
        return float(eff) if eff is not None else None, float(cost) if cost is not None else None, qcost

    # Fallback: lookup by name in ppu_df
    if name is None:
        return None, None, None
    row = ppu_df[ppu_df['PPU'] == name]
    if row.empty:
        return None, None, None
    eff = row['Efficiency'].iat[0] if 'Efficiency' in row.columns else None
    cost = row['Cost_CHF_per_kWh'].iat[0] if 'Cost_CHF_per_kWh' in row.columns else None
    if 'Quarter_CHF_per_kWh' in row.columns and pd.notna(row['Quarter_CHF_per_kWh'].iat[0]):
        qcost = float(row['Quarter_CHF_per_kWh'].iat[0])
    elif cost is not None:
        qcost = float(cost) * delta_t
    else:
        qcost = None
    return float(eff) if eff is not None else None, float(cost) if cost is not None else None, qcost

PPU 'Solar_Panel_1' added.
PPU 'Wind_Turbine_1' added.
PPU DataFrame:
              PPU   Type  Efficiency  Cost_CHF_per_kWh  quarter_CHF_per_kWh
0   Solar_Panel_1  solar        0.20              0.15                 0.25
1  Wind_Turbine_1   wind        0.35              0.12                 0.25

Storage DataFrame:
Empty DataFrame
Columns: [Storage, Capacity_MWh, Current_Value_MWh]
Index: []

Location DataFrame:
Empty DataFrame
Columns: [PPU, Type, Latitude, Longitude, Rank, Energy_Potential]
Index: []
